In [6]:
# Turn the code to function
import geopandas as gpd
from shapely.geometry import Point
from pyomo.environ import *

import random
import numpy as np
import pandas as pd

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

def MILP_pavement(budget, pavement_cost, points_payevemnt):
    # Define points dataframe
    points_df = pd.DataFrame(points_payevemnt, columns=['lat', 'lon'])

    # *****************************************read the shapefile (adjust the path to the shapefile)**********************
    shp = gpd.read_file("./grids/calgary_grids.shp")
    
    # create a geodataframe of the random points
    gdf = gpd.GeoDataFrame(points_df, geometry=[Point(x, y) for x, y in zip(points_df.lon, points_df.lat)])
    # spatial join
    join_dfs = gpd.sjoin(gdf, shp, op='within')
    
    # pivot the table to get the count of points in each grid cell
    pivot = join_dfs.pivot_table(index='id', aggfunc='size')

    # get the polygon of each grid cell from join_dfs
    polygons = join_dfs[['id', 'geometry']].drop_duplicates(subset='id')

    # get the id in pivot as Number of geographical regions
    N = len(pivot)

    # get the count of points in each grid cell as Initial distribution of pavement repairs
    pavement_distribution = pivot.values.tolist()

    # Number of Pavement types
    M = len(pavement_cost)

    # Create a ConcreteModel
    model = ConcreteModel()

    # Define variables
    model.x = Var(range(N), range(M), domain=Binary)  # Binary variable for repairing each pavement type in each region

    # Define objective function
    model.obj = Objective(expr=sum(model.x[n, m] for n in range(N) for m in range(M)), sense=maximize)

    # Constraints

    # Budget constraint
    model.budget_constraint = Constraint(expr=sum(pavement_cost[m] * pavement_distribution[n] * model.x[n, m] for n in range(N) for m in range(M)) <= budget)

    # Solve the model
    # check if glpk solver is not installed
    try:
        solver_result = SolverFactory('glpk').solve(model)
    except:
        print('glpk solver is not installed. Please install it first.')
        
    # Check the solver status
    if solver_result.solver.status == SolverStatus.ok:
        if solver_result.solver.termination_condition == TerminationCondition.optimal:
            print("Solver terminated successfully and found an optimal solution.")
        else:
            print("Solver terminated successfully but did not find an optimal solution.")
    elif solver_result.solver.status == SolverStatus.warning:
        print("Solver encountered a warning.")
    else:
        print("Solver encountered an error.")

    # get equvalent piviot id that coresponds to N
    pivot_id = pivot.index.tolist()

    # return selected pavement types in each region as a list with its index and overal regions in a dictionary along with total cost
    pavement_type = {}
    for n in range(N):
        region_repairs = []
        # get the id of the grid cell
        id = pivot_id[n]
        for m in range(M):
            if model.x[n, m].value == 1:
                region_repairs.append(m)
        pavement_type[id] = region_repairs
    # return the results
    return {'pavement_type': pavement_type, 'total_cost': sum(pavement_cost[m] * model.x[n, m].value for n in range(N) for m in range(M)), 'polygons': polygons}


In [7]:
# test the function

# generate a list of random lat and long coordinates
lat = np.random.uniform(low=51.0, high=51.2, size=(100,))   # 100 random lat coordinates    
lon = np.random.uniform(low=-115.2, high=-114.0, size=(100,)) # 100 random long coordinates
# create a list of lists containing lat and long coordinates
points = []
for i in range(len(lat)):
    points.append((lat[i], lon[i]))

# budget
budget = 50000

# pavement cost
pavement_cost = [1000, 1500, 1800, 2200]  # Cost for repairing each pavement type

MILP_pavement(budget, pavement_cost, points)

Solver terminated successfully and found an optimal solution.


{'pavement_type': {1.0: [0, 1],
  2.0: [0],
  3.0: [0],
  4.0: [0, 1],
  8.0: [0, 1, 2, 3],
  9.0: [],
  10.0: [0],
  11.0: [0],
  15.0: [0, 1, 2, 3],
  17.0: [0],
  18.0: [0, 1, 2, 3],
  22.0: [0, 1, 2, 3],
  24.0: []},
 'total_cost': 36000.0,
 'polygons':       id                     geometry
 1    1.0  POINT (-114.31512 51.16138)
 5    8.0  POINT (-114.18530 51.16959)
 6    3.0  POINT (-114.23770 51.09502)
 8    9.0  POINT (-114.14430 51.12861)
 11  24.0  POINT (-114.00696 51.08357)
 12  10.0  POINT (-114.16357 51.05427)
 15   4.0  POINT (-114.29084 51.03491)
 29  18.0  POINT (-114.12540 51.01483)
 33  11.0  POINT (-114.14609 51.03271)
 46   2.0  POINT (-114.24456 51.14455)
 58  17.0  POINT (-114.12406 51.06911)
 91  22.0  POINT (-114.02749 51.16786)
 99  15.0  POINT (-114.11156 51.16986)}

In [9]:
points

[(51.04564613051767, -114.59803419992188),
 (51.16137703525485, -114.31511719990041),
 (51.06969269586954, -115.00888217670465),
 (51.199150124710926, -114.98444752113177),
 (51.16702305402279, -115.09790918433566),
 (51.16958870958401, -114.18529918529785),
 (51.095019388253604, -114.23769641780807),
 (51.19026499191274, -114.70361898408574),
 (51.128610180713984, -114.14429676369483),
 (51.05135999989831, -114.93033483461268),
 (51.07876317981816, -114.64394368432357),
 (51.08356929064827, -114.00695907797774),
 (51.05426813644249, -114.16357263398811),
 (51.13168549728452, -114.63249782758798),
 (51.12012758452922, -114.84294741696604),
 (51.034911868479455, -114.29084258006125),
 (51.100296830725156, -115.0400780288622),
 (51.015203986140676, -114.6097015297754),
 (51.16669694103605, -114.94743092854209),
 (51.18392728220363, -114.87900445377791),
 (51.127702306232436, -114.2039257371237),
 (51.01271866412117, -114.31693843728337),
 (51.05131369028326, -114.6947175829941),
 (51.012